In [1]:
%%capture
!pip install dfcx-scrapi

## imports

In [3]:
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g


## g-sheets config

In [7]:
# service account scopes
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '18qvh0uWkA0z7S5CC3xTdvD2wkhsmnINjgxUWmuoCo7M'
sh = client.open_by_key(spreadsheet_key)
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad


## agent object attribute mappings


In [9]:
# object data attribute mappings 
# master workbook @ https://itrau.co/bc-dfcx-sad-main

project_id = sh.worksheet('Main').acell('B2').value
display_name = sh.worksheet('Main').acell('B3').value
gcp_region = sh.worksheet('Main').acell('B4').value
zone = sh.worksheet('Main').acell('B5').value
language = sh.worksheet('Main').acell('B6').value
default_start_flow = language = sh.worksheet('Main').acell('B8').value
time_zone = sh.worksheet('Main').acell('B7').value
creds_path = sh.worksheet('Main').acell('B8').value

print(project_id, display_name, gcp_region, gcp_region, zone, language, default_start_flow,time_zone)

bot1-404120  agent1 global global None  projects/bot1-404120/locations/global/agents/1f52228b-df94-4727-9207-61405b75906b/flows/00000000-0000-0000-0000-000000000000  projects/bot1-404120/locations/global/agents/1f52228b-df94-4727-9207-61405b75906b/flows/00000000-0000-0000-0000-000000000000  America/Chicago


In [ ]:
### endgoal is heatmap replication of https://itrau.co/heatmap